Iñigo Martin Melero

## Analisis SMOTE

### 0) Librerias

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from imblearn.over_sampling import SMOTE
from collections import Counter

from sklearn.metrics import accuracy_score
from sklearn.metrics import f1_score
from sklearn.metrics import cohen_kappa_score
from sklearn.metrics import confusion_matrix
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score

from sklearn.model_selection import cross_val_predict
from sklearn.model_selection import GridSearchCV
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn import tree
from sklearn.naive_bayes import GaussianNB
from sklearn.neural_network import MLPClassifier
from sklearn.svm import SVC
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.ensemble import GradientBoostingClassifier

np.random.seed(51)

### 1) Preprocesamiento

In [2]:
df1=pd.read_csv('Bancarrota_total_nd_nooutliers.csv',encoding = 'unicode_escape')
print(df1)

       ID  Coef1_Springate  Coef2_Springate  Coef3_Springate  Coef4_Springate  \
0       2           0.6894           0.1641           0.3116           2.8739   
1       4          -0.0034           0.0296          -0.2114           0.6265   
2       5           0.2916           0.0341           0.0534           0.3098   
3       6           0.3450          -0.1234          -0.1481           0.4249   
4       7           0.0593          -0.3758          -0.7022           0.0811   
..    ...              ...              ...              ...              ...   
837  1493          -0.3422          -0.1538          -0.2613           0.0100   
838  1496           0.0247          -0.1097          -0.0973           0.1505   
839  1497          -0.0206          -0.0668          -0.0340           0.0016   
840  1499           0.0964          -0.2809          -0.3393           0.0100   
841  1502          -0.0555          -0.3837          -0.3051           0.0354   

     Coef1_Amat  Coef2_Amat

In [3]:
df1.dtypes

ID                   int64
Coef1_Springate    float64
Coef2_Springate    float64
Coef3_Springate    float64
Coef4_Springate    float64
Coef1_Amat         float64
Coef2_Amat         float64
Coef3_Amat         float64
Coef4_Amat         float64
Resultado            int64
dtype: object

In [4]:
porcentaje_train=0.8

In [5]:
limitedf1=round(porcentaje_train*len(df1))
limitedf1

674

In [6]:
df1_train=df1[:limitedf1]
df1_train

,ID,Coef1_Springate,Coef2_Springate,Coef3_Springate,Coef4_Springate,Coef1_Amat,Coef2_Amat,Coef3_Amat,Coef4_Amat,Resultado
0,2,0.6894,0.1641,0.3116,2.8739,1.7204,0.4234,0.1196,0.2824,1
1,4,-0.0034,0.0296,-0.2114,0.6265,0.4756,0.2109,-0.1652,-0.7833,1
2,5,0.2916,0.0341,0.0534,0.3098,0.9662,0.3709,0.0299,0.0807,1
3,6,0.3450,-0.1234,-0.1481,0.4249,0.7613,-0.3175,-0.1566,0.4933,1
4,7,0.0593,-0.3758,-0.7022,0.0811,0.1077,-0.2699,-0.3977,1.4737,1
...,...,...,...,...,...,...,...,...,...,...
669,1071,-0.0003,-0.0624,-0.1357,0.0415,0.0003,0.2014,-0.0634,-0.3148,1
670,1072,0.4101,-0.2810,-0.8405,0.4119,1.5439,0.3125,-0.2256,-0.7217,1
671,1073,-0.4541,-0.0913,-0.1203,0.1065,0.9844,0.0554,-0.0718,-1.2955,1
672,1074,-0.0797,0.1413,0.6807,0.8580,3.5118,-1.1137,0.0854,-0.0767,1


In [7]:
df1_test=df1[limitedf1:]
df1_test

,ID,Coef1_Springate,Coef2_Springate,Coef3_Springate,Coef4_Springate,Coef1_Amat,Coef2_Amat,Coef3_Amat,Coef4_Amat,Resultado
674,1076,0.6084,-0.3628,-0.2173,0.5857,0.4065,-0.6763,-0.2731,0.4039,1
675,1078,0.1225,0.0733,0.1669,2.6911,1.1318,0.5834,0.0469,0.0804,1
676,1079,0.0473,-0.0602,-0.1312,0.0743,0.2080,0.1653,-0.0751,-0.4541,1
677,1080,-0.0577,-0.1091,-0.3469,0.6132,2.3731,0.2095,-0.1095,-0.5226,1
678,1081,0.0582,0.0702,0.0354,0.7831,0.6438,0.5311,0.0156,0.0293,1
...,...,...,...,...,...,...,...,...,...,...
837,1493,-0.3422,-0.1538,-0.2613,0.0100,0.1495,-0.2978,-0.1422,0.4773,1
838,1496,0.0247,-0.1097,-0.0973,0.1505,0.0879,-0.1280,-0.0823,0.6432,1
839,1497,-0.0206,-0.0668,-0.0340,0.0016,0.2716,-1.1209,-0.0501,0.0447,1
840,1499,0.0964,-0.2809,-0.3393,0.0100,0.5816,-0.1982,-0.2204,1.1124,1


In [8]:
df2=pd.read_csv('Activa_total_nd_nooutliers.csv',encoding = 'unicode_escape')
print(df2)

            ID  Coef1_Springate  Coef2_Springate  Coef3_Springate  \
0            1          -0.1786           0.0810           0.2086   
1            2           0.3473           0.0039           0.1439   
2            3           0.4718           0.0385           0.0752   
3            5           0.0656           0.0826           0.5767   
4            7           0.3031           0.0006           0.0030   
...        ...              ...              ...              ...   
388140  732492           0.0111          -0.0076          -0.0069   
388141  732493           0.8986          -0.0367          -0.2176   
388142  732499           0.0051          -0.0130          -0.0203   
388143  732550          -0.0927           0.0000           0.0208   
388144  732570           0.6813          -0.0064          -0.0240   

        Coef4_Springate  Coef1_Amat  Coef2_Amat  Coef3_Amat  Coef4_Amat  \
0                2.3157      0.9028      0.6267      0.0598      0.0954   
1                1.89

In [9]:
df2.dtypes

ID                   int64
Coef1_Springate    float64
Coef2_Springate    float64
Coef3_Springate    float64
Coef4_Springate    float64
Coef1_Amat         float64
Coef2_Amat         float64
Coef3_Amat         float64
Coef4_Amat         float64
Resultado            int64
dtype: object

In [10]:
limitedf2=round(porcentaje_train*len(df2))
limitedf2

310516

In [11]:
df2_train=df2[:limitedf2]
df2_train

,ID,Coef1_Springate,Coef2_Springate,Coef3_Springate,Coef4_Springate,Coef1_Amat,Coef2_Amat,Coef3_Amat,Coef4_Amat,Resultado
0,1,-0.1786,0.0810,0.2086,2.3157,0.9028,0.6267,0.0598,0.0954,2
1,2,0.3473,0.0039,0.1439,1.8919,1.0684,0.2626,0.0774,0.2946,2
2,3,0.4718,0.0385,0.0752,2.1073,1.4550,0.4008,0.0199,0.0498,2
3,5,0.0656,0.0826,0.5767,0.6623,2.1201,0.8463,0.0806,0.0953,2
4,7,0.3031,0.0006,0.0030,1.0532,1.1328,0.0219,0.0013,0.0573,2
...,...,...,...,...,...,...,...,...,...,...
310511,510759,0.1624,0.0324,0.0028,0.9055,1.9388,0.3441,-0.0120,-0.0349,2
310512,510761,0.5481,0.0658,0.1927,2.4296,2.6813,0.6583,0.0494,0.0750,2
310513,510762,0.2481,0.0123,0.0318,2.4477,3.0912,0.5524,0.0059,0.0106,2
310514,510764,0.0755,0.0939,1.1422,0.7479,6.4856,0.5255,0.0613,0.1165,2


In [12]:
df2_test=df2[limitedf2:]
df2_test

,ID,Coef1_Springate,Coef2_Springate,Coef3_Springate,Coef4_Springate,Coef1_Amat,Coef2_Amat,Coef3_Amat,Coef4_Amat,Resultado
310516,510768,0.1113,0.0026,0.0032,4.0011,1.2416,0.2112,0.0019,0.0091,2
310517,510770,-0.0817,0.0541,0.1332,3.7004,1.6666,0.5993,0.0343,0.0572,2
310518,510771,0.0540,0.0084,0.3318,0.3122,6.0255,0.9748,0.0063,0.0064,2
310519,510782,-0.1438,0.1316,0.2109,0.8806,0.8229,0.3638,0.1316,0.3618,2
310520,510783,0.5868,0.0284,0.0602,0.1589,1.6683,0.5282,0.0213,0.0404,2
...,...,...,...,...,...,...,...,...,...,...
388140,732492,0.0111,-0.0076,-0.0069,0.0000,0.9596,-0.0421,-0.0072,0.1714,2
388141,732493,0.8986,-0.0367,-0.2176,0.0000,3.7207,-0.1153,-0.0585,0.5071,2
388142,732499,0.0051,-0.0130,-0.0203,0.0000,0.0189,0.3601,-0.0130,-0.0360,2
388143,732550,-0.0927,0.0000,0.0208,0.0000,1.0615,0.4367,0.0117,0.0269,2


### 2) Preparacion datos para ML

In [13]:
df3_train=pd.concat([df1_train, df2_train])
X_train=df3_train.drop(columns=["Resultado","ID"])
Y_train=df3_train.drop(columns=["ID","Coef1_Amat","Coef2_Amat","Coef3_Amat","Coef4_Amat","Coef1_Springate","Coef2_Springate","Coef3_Springate","Coef4_Springate"])

In [14]:
sm=SMOTE(random_state=42)
X_SMOTE,Y_SMOTE=sm.fit_resample(X_train,Y_train)
df3_train_SMOTE=pd.concat([X_SMOTE, Y_SMOTE],axis=1)
df3_train_SMOTE = df3_train_SMOTE.sample(frac=1).reset_index(drop=True)
df3_train_SMOTE

,Coef1_Springate,Coef2_Springate,Coef3_Springate,Coef4_Springate,Coef1_Amat,Coef2_Amat,Coef3_Amat,Coef4_Amat,Resultado
0,0.033857,0.005717,-0.415464,0.245682,0.332314,0.265841,-0.089677,-0.341928,1
1,0.593800,0.002800,-0.049100,2.051700,2.305900,0.136100,-0.020900,-0.153900,2
2,0.436200,0.135000,0.361900,2.504800,2.082800,0.404400,0.212800,0.526300,2
3,0.055439,0.022579,0.021515,0.710773,1.405784,0.151975,0.007902,0.061273,1
4,0.506600,0.071000,0.158000,0.693100,1.934100,0.451600,0.044300,0.098200,2
...,...,...,...,...,...,...,...,...,...
621027,0.265400,0.080200,0.205000,1.190100,2.280600,0.569900,0.082400,0.144500,2
621028,0.549700,0.062200,0.140600,1.832100,2.615500,0.303300,0.049700,0.164100,2
621029,0.127641,0.045261,0.043149,0.716546,2.243597,0.520874,0.013702,0.027610,1
621030,-0.027800,0.216200,0.598200,1.808800,1.951300,0.385200,0.188400,0.489000,2


In [15]:
df3_test=pd.concat([df1_test, df2_test])
X_test=df3_test.drop(columns=["Resultado","ID"])
Y_test=df3_test.drop(columns=["ID","Coef1_Amat","Coef2_Amat","Coef3_Amat","Coef4_Amat","Coef1_Springate","Coef2_Springate","Coef3_Springate","Coef4_Springate"])

In [16]:
df3_test_SMOTE=pd.concat([X_test, Y_test],axis=1)

In [17]:
df3_total_SMOTE=pd.concat([df3_train_SMOTE, df3_test_SMOTE])
df3_total_SMOTE

,Coef1_Springate,Coef2_Springate,Coef3_Springate,Coef4_Springate,Coef1_Amat,Coef2_Amat,Coef3_Amat,Coef4_Amat,Resultado
0,0.033857,0.005717,-0.415464,0.245682,0.332314,0.265841,-0.089677,-0.341928,1
1,0.593800,0.002800,-0.049100,2.051700,2.305900,0.136100,-0.020900,-0.153900,2
2,0.436200,0.135000,0.361900,2.504800,2.082800,0.404400,0.212800,0.526300,2
3,0.055439,0.022579,0.021515,0.710773,1.405784,0.151975,0.007902,0.061273,1
4,0.506600,0.071000,0.158000,0.693100,1.934100,0.451600,0.044300,0.098200,2
...,...,...,...,...,...,...,...,...,...
388140,0.011100,-0.007600,-0.006900,0.000000,0.959600,-0.042100,-0.007200,0.171400,2
388141,0.898600,-0.036700,-0.217600,0.000000,3.720700,-0.115300,-0.058500,0.507100,2
388142,0.005100,-0.013000,-0.020300,0.000000,0.018900,0.360100,-0.013000,-0.036000,2
388143,-0.092700,0.000000,0.020800,0.000000,1.061500,0.436700,0.011700,0.026900,2


In [18]:
X_train_SMOTE=df3_train_SMOTE.drop(columns=["Resultado"])
Y_train_SMOTE=df3_train_SMOTE.drop(columns=["Coef1_Amat","Coef2_Amat","Coef3_Amat","Coef4_Amat","Coef1_Springate","Coef2_Springate","Coef3_Springate","Coef4_Springate"])
X_test_SMOTE=df3_test_SMOTE.drop(columns=["Resultado"])
Y_test_SMOTE=df3_test_SMOTE.drop(columns=["Coef1_Amat","Coef2_Amat","Coef3_Amat","Coef4_Amat","Coef1_Springate","Coef2_Springate","Coef3_Springate","Coef4_Springate"])

### 3.1) Valores Test de Springate

Calculamos si está comprometida la situación de la empresa según Amat.

Su formula es:

Z = - 0.862 + 1.03 x Coef1 + 3.07 x Coef2 + 0.66 x Coef3 + 0.4 x Coef4

Z positivo empresa bien, Z negativo empresa mal.

Cogemos otro Dataframe, y multiplicamos primero y luego sumamos.

In [19]:
formula_Spr=X_test_SMOTE
formula_Spr=formula_Spr.drop(columns=["Coef1_Amat","Coef2_Amat","Coef3_Amat","Coef4_Amat"])
formula_Spr.loc[:,'Coef1_Springate'] *= 1.03
formula_Spr.loc[:,'Coef2_Springate'] *= 3.07
formula_Spr.loc[:,'Coef3_Springate'] *= 0.66
formula_Spr.loc[:,'Coef4_Springate'] *= 0.4
formula_Spr["constant"]=-0.862
formula_Spr["Z"] = formula_Spr.sum(axis=1)

formula_Spr["Y_pred_Springate"] = 2
formula_Spr.loc[formula_Spr["Z"] <= 0, "Y_pred_Springate"] = 1
Y_pred_formula_Spr=formula_Spr["Y_pred_Springate"]
print("Accuracy: "+str(accuracy_score(Y_test_SMOTE, Y_pred_formula_Spr)))
print("F1 score: "+str(f1_score(Y_test_SMOTE, Y_pred_formula_Spr)))
print("Kappa: "+str(cohen_kappa_score(Y_test_SMOTE, Y_pred_formula_Spr)))
print("Confusion matrix:")
confusion_matrix(Y_test_SMOTE, Y_pred_formula_Spr)

Accuracy: 0.3064899674794658
F1 score: 0.005199594357886974
Kappa: 0.0008979061931188026
Confusion matrix:


array([[  141,    27],
       [53926, 23703]], dtype=int64)

### 3.2) Valores Test de Grover

Su formula es:

Z = + 0.057 + 0.02 + 1.650 x Coef1 + 3.404 x Coef2 - 0.016 x Coef3

Z positivo empresa bien, Z negativo empresa mal.

Cogemos otro Dataframe, y multiplicamos primero y luego sumamos.

In [20]:
formula_Gro=X_test_SMOTE
formula_Gro=formula_Gro.drop(columns=["Coef1_Amat","Coef2_Amat","Coef3_Springate","Coef4_Springate","Coef4_Amat"])
formula_Gro.loc[:,'Coef1_Springate'] *= 1.65
formula_Gro.loc[:,'Coef2_Springate'] *= 3.404
formula_Gro.loc[:,'Coef3_Amat'] *= -0.016
formula_Gro["constant"]=0.077
formula_Gro["Z"] = formula_Gro.sum(axis=1)

formula_Gro["Y_pred_Grover"] = 2
formula_Gro.loc[formula_Gro["Z"] <= 0, "Y_pred_Grover"] = 1
Y_pred_formula_Gro=formula_Gro["Y_pred_Grover"]
print("Accuracy: "+str(accuracy_score(Y_test_SMOTE, Y_pred_formula_Gro)))
print("F1 score: "+str(f1_score(Y_test_SMOTE, Y_pred_formula_Gro)))
print("Kappa: "+str(cohen_kappa_score(Y_test_SMOTE, Y_pred_formula_Gro)))
print("Confusion matrix:")
confusion_matrix(Y_test_SMOTE, Y_pred_formula_Gro)

Accuracy: 0.902721184621515
F1 score: 0.01867219917012448
Kappa: 0.01450864802718177
Confusion matrix:


array([[   72,    96],
       [ 7472, 70157]], dtype=int64)

### 3.3) Valores Test de Amat

Calculamos si está comprometida la situación de la empresa según Amat.

Su formula es:

Z = - 3.9 + 1.28 x Coef1 + 6.1 x Coef2 + 6.5 x Coef3 + 4.8 x Coef4

Z positivo empresa bien, Z negativo empresa mal.

Cogemos otro Dataframe, y multiplicamos primero y luego sumamos.

In [21]:
formula_Ama=X_test_SMOTE
formula_Ama=formula_Ama.drop(columns=["Coef1_Springate","Coef2_Springate","Coef3_Springate","Coef4_Springate"])
formula_Ama.loc[:,'Coef1_Amat'] *= 1.28
formula_Ama.loc[:,'Coef2_Amat'] *= 6.1
formula_Ama.loc[:,'Coef3_Amat'] *= 6.5
formula_Ama.loc[:,'Coef4_Amat'] *= 4.8
formula_Ama["constant"]=-3.9
formula_Ama["Z"] = formula_Ama.sum(axis=1)

formula_Ama["Y_pred_Amat"] = 2
formula_Ama.loc[formula_Ama["Z"] <= 0, "Y_pred_Amat"] = 1
Y_pred_formula_Ama=formula_Ama["Y_pred_Amat"]
print("Accuracy: "+str(accuracy_score(Y_test_SMOTE, Y_pred_formula_Ama)))
print("F1 score: "+str(f1_score(Y_test_SMOTE, Y_pred_formula_Ama)))
print("Kappa: "+str(cohen_kappa_score(Y_test_SMOTE, Y_pred_formula_Ama)))
print("Confusion matrix:")
confusion_matrix(Y_test_SMOTE, Y_pred_formula_Ama)

Accuracy: 0.6004087561217013
F1 score: 0.007787813986147905
Kappa: 0.0035071056677364876
Confusion matrix:


array([[  122,    46],
       [31041, 46588]], dtype=int64)

### 3.4) Combinado

In [22]:
labels_total=pd.concat([Y_test_SMOTE,Y_pred_formula_Spr, Y_pred_formula_Gro,Y_pred_formula_Ama], axis=1)
print(labels_total.to_string())

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



In [23]:
bancarrota=labels_total.loc[(labels_total['Resultado']==1)]

In [24]:
at_least_1_bancarrota=sum(bancarrota.sum(axis=1)<7)
print(at_least_1_bancarrota)
all_match_bancarrota=sum(bancarrota.sum(axis=1)==4)
print(all_match_bancarrota)

153
65


### 4) Modelos ML

### 4.1) Estudio con Springate

In [25]:
X_train_SMOTE_Springate=X_train_SMOTE.drop(columns=["Coef1_Amat","Coef2_Amat","Coef3_Amat","Coef4_Amat"])
X_train_SMOTE_Springate

,Coef1_Springate,Coef2_Springate,Coef3_Springate,Coef4_Springate
0,0.033857,0.005717,-0.415464,0.245682
1,0.593800,0.002800,-0.049100,2.051700
2,0.436200,0.135000,0.361900,2.504800
3,0.055439,0.022579,0.021515,0.710773
4,0.506600,0.071000,0.158000,0.693100
...,...,...,...,...
621027,0.265400,0.080200,0.205000,1.190100
621028,0.549700,0.062200,0.140600,1.832100
621029,0.127641,0.045261,0.043149,0.716546
621030,-0.027800,0.216200,0.598200,1.808800


In [26]:
X_test_SMOTE_Springate=X_test_SMOTE.drop(columns=["Coef1_Amat","Coef2_Amat","Coef3_Amat","Coef4_Amat"])
X_test_SMOTE_Springate

,Coef1_Springate,Coef2_Springate,Coef3_Springate,Coef4_Springate
674,0.6084,-0.3628,-0.2173,0.5857
675,0.1225,0.0733,0.1669,2.6911
676,0.0473,-0.0602,-0.1312,0.0743
677,-0.0577,-0.1091,-0.3469,0.6132
678,0.0582,0.0702,0.0354,0.7831
...,...,...,...,...
388140,0.0111,-0.0076,-0.0069,0.0000
388141,0.8986,-0.0367,-0.2176,0.0000
388142,0.0051,-0.0130,-0.0203,0.0000
388143,-0.0927,0.0000,0.0208,0.0000


K Nearest Neighbours

In [27]:
knn = KNeighborsClassifier()
knn.fit(X_train_SMOTE_Springate, Y_train_SMOTE)
y_pred = knn.predict(X_test_SMOTE_Springate)

C:\Users\inima\anaconda3\envs\lamda_ssl\lib\site-packages\sklearn\neighbors\_classification.py:198: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return self._fit(X, y)


In [28]:
print("Accuracy: "+str(accuracy_score(Y_test_SMOTE, y_pred)))
print("F1 score: "+str(f1_score(Y_test_SMOTE, y_pred)))
print("Kappa: "+str(cohen_kappa_score(Y_test_SMOTE, y_pred)))
print("Confusion matrix:")
confusion_matrix(Y_test_SMOTE, y_pred)

Accuracy: 0.8886075298533362
F1 score: 0.016568315932818885
Kappa: 0.01238419016165393
Confusion matrix:


array([[   73,    95],
       [ 8571, 69058]], dtype=int64)

Random Forest

In [29]:
rf = RandomForestClassifier()
rf.fit(X_train_SMOTE_Springate, Y_train_SMOTE)
y_pred = rf.predict(X_test_SMOTE_Springate)

C:\Users\inima\anaconda3\envs\lamda_ssl\lib\site-packages\ipykernel_launcher.py:2: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  


In [30]:
print("Accuracy: "+str(accuracy_score(Y_test_SMOTE, y_pred)))
print("F1 score: "+str(f1_score(Y_test_SMOTE, y_pred)))
print("Kappa: "+str(cohen_kappa_score(Y_test_SMOTE, y_pred)))
print("Confusion matrix:")
confusion_matrix(Y_test_SMOTE, y_pred)

Accuracy: 0.9622993174544006
F1 score: 0.044936502767828075
Kappa: 0.04102131494704364
Confusion matrix:


array([[   69,    99],
       [ 2834, 74795]], dtype=int64)

Decision Forest

In [31]:
clf = tree.DecisionTreeClassifier()
clf.fit(X_train_SMOTE_Springate, Y_train_SMOTE)
y_pred = clf.predict(X_test_SMOTE_Springate)

In [32]:
print("Accuracy: "+str(accuracy_score(Y_test_SMOTE, y_pred)))
print("F1 score: "+str(f1_score(Y_test_SMOTE, y_pred)))
print("Kappa: "+str(cohen_kappa_score(Y_test_SMOTE, y_pred)))
print("Confusion matrix:")
confusion_matrix(Y_test_SMOTE, y_pred)

Accuracy: 0.9430055143514532
F1 score: 0.03018372703412073
Kappa: 0.02613221141671107
Confusion matrix:


array([[   69,    99],
       [ 4335, 73294]], dtype=int64)

Naive Bayes

In [33]:
gnb = GaussianNB()
gnb.fit(X_train_SMOTE_Springate, Y_train_SMOTE)
y_pred = gnb.predict(X_test_SMOTE_Springate)

C:\Users\inima\anaconda3\envs\lamda_ssl\lib\site-packages\sklearn\utils\validation.py:993: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


In [34]:
print("Accuracy: "+str(accuracy_score(Y_test_SMOTE, y_pred)))
print("F1 score: "+str(f1_score(Y_test_SMOTE, y_pred)))
print("Kappa: "+str(cohen_kappa_score(Y_test_SMOTE, y_pred)))
print("Confusion matrix:")
confusion_matrix(Y_test_SMOTE, y_pred)

Accuracy: 0.8289137113256295
F1 score: 0.014949674363528715
Kappa: 0.01073020758348342
Confusion matrix:


array([[  101,    67],
       [13243, 64386]], dtype=int64)

Multi-layer Perceptron Classifier Neural newtork

In [35]:
mlp = MLPClassifier()
mlp.fit(X_train_SMOTE_Springate, Y_train_SMOTE)
y_pred = mlp.predict(X_test_SMOTE_Springate)

C:\Users\inima\anaconda3\envs\lamda_ssl\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:1109: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\Users\inima\anaconda3\envs\lamda_ssl\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:696: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  ConvergenceWarning,


In [36]:
print("Accuracy: "+str(accuracy_score(Y_test_SMOTE, y_pred)))
print("F1 score: "+str(f1_score(Y_test_SMOTE, y_pred)))
print("Kappa: "+str(cohen_kappa_score(Y_test_SMOTE, y_pred)))
print("Confusion matrix:")
confusion_matrix(Y_test_SMOTE, y_pred)

Accuracy: 0.6585986606167333
F1 score: 0.009324878776575906
Kappa: 0.005054704104210939
Confusion matrix:


array([[  125,    43],
       [26517, 51112]], dtype=int64)

Linear Discriminant Analysis

In [37]:
lda = LinearDiscriminantAnalysis()
lda.fit(X_train_SMOTE_Springate, Y_train_SMOTE)
y_pred = lda.predict(X_test_SMOTE_Springate)

C:\Users\inima\anaconda3\envs\lamda_ssl\lib\site-packages\sklearn\utils\validation.py:993: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


In [38]:
print("Accuracy: "+str(accuracy_score(Y_test_SMOTE, y_pred)))
print("F1 score: "+str(f1_score(Y_test_SMOTE, y_pred)))
print("Kappa: "+str(cohen_kappa_score(Y_test_SMOTE, y_pred)))
print("Confusion matrix:")
confusion_matrix(Y_test_SMOTE, y_pred)

Accuracy: 0.6345231821278455
F1 score: 0.008439407149084569
Kappa: 0.004163655331926863
Confusion matrix:


array([[  121,    47],
       [28386, 49243]], dtype=int64)

## 4.2) Estudio con Grover

In [39]:
X_train_SMOTE_Grover=X_train_SMOTE.drop(columns=["Coef1_Amat","Coef2_Amat","Coef3_Springate","Coef4_Springate","Coef4_Amat"])
X_train_SMOTE_Grover

,Coef1_Springate,Coef2_Springate,Coef3_Amat
0,0.033857,0.005717,-0.089677
1,0.593800,0.002800,-0.020900
2,0.436200,0.135000,0.212800
3,0.055439,0.022579,0.007902
4,0.506600,0.071000,0.044300
...,...,...,...
621027,0.265400,0.080200,0.082400
621028,0.549700,0.062200,0.049700
621029,0.127641,0.045261,0.013702
621030,-0.027800,0.216200,0.188400


In [40]:
X_test_SMOTE_Grover=X_test_SMOTE.drop(columns=["Coef1_Amat","Coef2_Amat","Coef3_Springate","Coef4_Springate","Coef4_Amat"])
X_test_SMOTE_Grover

,Coef1_Springate,Coef2_Springate,Coef3_Amat
674,0.6084,-0.3628,-0.2731
675,0.1225,0.0733,0.0469
676,0.0473,-0.0602,-0.0751
677,-0.0577,-0.1091,-0.1095
678,0.0582,0.0702,0.0156
...,...,...,...
388140,0.0111,-0.0076,-0.0072
388141,0.8986,-0.0367,-0.0585
388142,0.0051,-0.0130,-0.0130
388143,-0.0927,0.0000,0.0117


K Nearest Neighbours

In [41]:
knn = KNeighborsClassifier()
knn.fit(X_train_SMOTE_Grover, Y_train_SMOTE)
y_pred = knn.predict(X_test_SMOTE_Grover)

C:\Users\inima\anaconda3\envs\lamda_ssl\lib\site-packages\sklearn\neighbors\_classification.py:198: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return self._fit(X, y)


In [42]:
print("Accuracy: "+str(accuracy_score(Y_test_SMOTE, y_pred)))
print("F1 score: "+str(f1_score(Y_test_SMOTE, y_pred)))
print("Kappa: "+str(cohen_kappa_score(Y_test_SMOTE, y_pred)))
print("Confusion matrix:")
confusion_matrix(Y_test_SMOTE, y_pred)

Accuracy: 0.8058151342596758
F1 score: 0.012291598561621445
Kappa: 0.00805450981822764
Confusion matrix:


array([[   94,    74],
       [15033, 62596]], dtype=int64)

Random Forest

In [43]:
rf = RandomForestClassifier()
rf.fit(X_train_SMOTE_Grover, Y_train_SMOTE)
y_pred = rf.predict(X_test_SMOTE_Grover)

C:\Users\inima\anaconda3\envs\lamda_ssl\lib\site-packages\ipykernel_launcher.py:2: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  


In [44]:
print("Accuracy: "+str(accuracy_score(Y_test_SMOTE, y_pred)))
print("F1 score: "+str(f1_score(Y_test_SMOTE, y_pred)))
print("Kappa: "+str(cohen_kappa_score(Y_test_SMOTE, y_pred)))
print("Confusion matrix:")
confusion_matrix(Y_test_SMOTE, y_pred)

Accuracy: 0.902129902181318
F1 score: 0.023846153846153847
Kappa: 0.01970350965082379
Confusion matrix:


array([[   93,    75],
       [ 7539, 70090]], dtype=int64)

Decision Forest

In [45]:
clf = tree.DecisionTreeClassifier()
clf.fit(X_train_SMOTE_Grover, Y_train_SMOTE)
y_pred = clf.predict(X_test_SMOTE_Grover)

In [46]:
print("Accuracy: "+str(accuracy_score(Y_test_SMOTE, y_pred)))
print("F1 score: "+str(f1_score(Y_test_SMOTE, y_pred)))
print("Kappa: "+str(cohen_kappa_score(Y_test_SMOTE, y_pred)))
print("Confusion matrix:")
confusion_matrix(Y_test_SMOTE, y_pred)

Accuracy: 0.8648019846523645
F1 score: 0.01627384960718294
Kappa: 0.012074106897530634
Confusion matrix:


array([[   87,    81],
       [10437, 67192]], dtype=int64)

Naive Bayes

In [47]:
gnb = GaussianNB()
gnb.fit(X_train_SMOTE_Grover, Y_train_SMOTE)
y_pred = gnb.predict(X_test_SMOTE_Grover)

C:\Users\inima\anaconda3\envs\lamda_ssl\lib\site-packages\sklearn\utils\validation.py:993: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


In [48]:
print("Accuracy: "+str(accuracy_score(Y_test_SMOTE, y_pred)))
print("F1 score: "+str(f1_score(Y_test_SMOTE, y_pred)))
print("Kappa: "+str(cohen_kappa_score(Y_test_SMOTE, y_pred)))
print("Confusion matrix:")
confusion_matrix(Y_test_SMOTE, y_pred)

Accuracy: 0.8786971219969921
F1 score: 0.019736158720265917
Kappa: 0.01555861944383008
Confusion matrix:


array([[   95,    73],
       [ 9364, 68265]], dtype=int64)

Multi-layer Perceptron Classifier Neural newtork

In [49]:
mlp = MLPClassifier()
mlp.fit(X_train_SMOTE_Grover, Y_train_SMOTE)
y_pred = mlp.predict(X_test_SMOTE_Grover)

C:\Users\inima\anaconda3\envs\lamda_ssl\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:1109: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


In [50]:
print("Accuracy: "+str(accuracy_score(Y_test_SMOTE, y_pred)))
print("F1 score: "+str(f1_score(Y_test_SMOTE, y_pred)))
print("Kappa: "+str(cohen_kappa_score(Y_test_SMOTE, y_pred)))
print("Confusion matrix:")
confusion_matrix(Y_test_SMOTE, y_pred)

Accuracy: 0.7381775646875843
F1 score: 0.01183719012273808
Kappa: 0.007585953862180861
Confusion matrix:


array([[  122,    46],
       [20323, 57306]], dtype=int64)

Linear Discriminant Analysis

In [51]:
lda = LinearDiscriminantAnalysis()
lda.fit(X_train_SMOTE_Grover, Y_train_SMOTE)
y_pred = lda.predict(X_test_SMOTE_Grover)

C:\Users\inima\anaconda3\envs\lamda_ssl\lib\site-packages\sklearn\utils\validation.py:993: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


In [52]:
print("Accuracy: "+str(accuracy_score(Y_test_SMOTE, y_pred)))
print("F1 score: "+str(f1_score(Y_test_SMOTE, y_pred)))
print("Kappa: "+str(cohen_kappa_score(Y_test_SMOTE, y_pred)))
print("Confusion matrix:")
confusion_matrix(Y_test_SMOTE, y_pred)

Accuracy: 0.8041569726339062
F1 score: 0.013851132686084141
Kappa: 0.009620260689928717
Confusion matrix:


array([[  107,    61],
       [15175, 62454]], dtype=int64)

## 4.3) Estudio con Amat

In [53]:
X_train_SMOTE_Amat=X_train_SMOTE.drop(columns=["Coef1_Springate","Coef2_Springate","Coef3_Springate","Coef4_Springate"])
X_train_SMOTE_Amat

,Coef1_Amat,Coef2_Amat,Coef3_Amat,Coef4_Amat
0,0.332314,0.265841,-0.089677,-0.341928
1,2.305900,0.136100,-0.020900,-0.153900
2,2.082800,0.404400,0.212800,0.526300
3,1.405784,0.151975,0.007902,0.061273
4,1.934100,0.451600,0.044300,0.098200
...,...,...,...,...
621027,2.280600,0.569900,0.082400,0.144500
621028,2.615500,0.303300,0.049700,0.164100
621029,2.243597,0.520874,0.013702,0.027610
621030,1.951300,0.385200,0.188400,0.489000


In [54]:
X_test_SMOTE_Amat=X_test_SMOTE.drop(columns=["Coef1_Springate","Coef2_Springate","Coef3_Springate","Coef4_Springate"])
X_test_SMOTE_Amat

,Coef1_Amat,Coef2_Amat,Coef3_Amat,Coef4_Amat
674,0.4065,-0.6763,-0.2731,0.4039
675,1.1318,0.5834,0.0469,0.0804
676,0.2080,0.1653,-0.0751,-0.4541
677,2.3731,0.2095,-0.1095,-0.5226
678,0.6438,0.5311,0.0156,0.0293
...,...,...,...,...
388140,0.9596,-0.0421,-0.0072,0.1714
388141,3.7207,-0.1153,-0.0585,0.5071
388142,0.0189,0.3601,-0.0130,-0.0360
388143,1.0615,0.4367,0.0117,0.0269


K Nearest Neighbours

In [55]:
knn = KNeighborsClassifier()
knn.fit(X_train_SMOTE_Amat, Y_train_SMOTE)
y_pred = knn.predict(X_test_SMOTE_Amat)

C:\Users\inima\anaconda3\envs\lamda_ssl\lib\site-packages\sklearn\neighbors\_classification.py:198: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return self._fit(X, y)


In [56]:
print("Accuracy: "+str(accuracy_score(Y_test_SMOTE, y_pred)))
print("F1 score: "+str(f1_score(Y_test_SMOTE, y_pred)))
print("Kappa: "+str(cohen_kappa_score(Y_test_SMOTE, y_pred)))
print("Confusion matrix:")
confusion_matrix(Y_test_SMOTE, y_pred)

Accuracy: 0.8958057508644293
F1 score: 0.022666988184229565
Kappa: 0.01851387872907806
Confusion matrix:


array([[   94,    74],
       [ 8032, 69597]], dtype=int64)

Random Forest

In [57]:
rf = RandomForestClassifier()
rf.fit(X_train_SMOTE_Amat, Y_train_SMOTE)
y_pred = rf.predict(X_test_SMOTE_Amat)

C:\Users\inima\anaconda3\envs\lamda_ssl\lib\site-packages\ipykernel_launcher.py:2: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  


In [58]:
print("Accuracy: "+str(accuracy_score(Y_test_SMOTE, y_pred)))
print("F1 score: "+str(f1_score(Y_test_SMOTE, y_pred)))
print("Kappa: "+str(cohen_kappa_score(Y_test_SMOTE, y_pred)))
print("Confusion matrix:")
confusion_matrix(Y_test_SMOTE, y_pred)

Accuracy: 0.9651400439605641
F1 score: 0.06288873531444367
Kappa: 0.05906079359677696
Confusion matrix:


array([[   91,    77],
       [ 2635, 74994]], dtype=int64)

Decision Forest

In [59]:
clf = tree.DecisionTreeClassifier()
clf.fit(X_train_SMOTE_Amat, Y_train_SMOTE)
y_pred = clf.predict(X_test_SMOTE_Amat)

In [60]:
print("Accuracy: "+str(accuracy_score(Y_test_SMOTE, y_pred)))
print("F1 score: "+str(f1_score(Y_test_SMOTE, y_pred)))
print("Kappa: "+str(cohen_kappa_score(Y_test_SMOTE, y_pred)))
print("Confusion matrix:")
confusion_matrix(Y_test_SMOTE, y_pred)

Accuracy: 0.9505636464131008
F1 score: 0.045657568238213396
Kappa: 0.041691235810944915
Confusion matrix:


array([[   92,    76],
       [ 3770, 73859]], dtype=int64)

Naive Bayes

In [61]:
gnb = GaussianNB()
gnb.fit(X_train_SMOTE_Amat, Y_train_SMOTE)
y_pred = gnb.predict(X_test_SMOTE_Amat)

C:\Users\inima\anaconda3\envs\lamda_ssl\lib\site-packages\sklearn\utils\validation.py:993: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


In [62]:
print("Accuracy: "+str(accuracy_score(Y_test_SMOTE, y_pred)))
print("F1 score: "+str(f1_score(Y_test_SMOTE, y_pred)))
print("Kappa: "+str(cohen_kappa_score(Y_test_SMOTE, y_pred)))
print("Confusion matrix:")
confusion_matrix(Y_test_SMOTE, y_pred)

Accuracy: 0.8149285962183632
F1 score: 0.016395682470282825
Kappa: 0.012178311806791853
Confusion matrix:


array([[  120,    48],
       [14350, 63279]], dtype=int64)

Multi-layer Perceptron Classifier Neural newtork

In [63]:
mlp = MLPClassifier()
mlp.fit(X_train_SMOTE_Amat, Y_train_SMOTE)
y_pred = mlp.predict(X_test_SMOTE_Amat)

C:\Users\inima\anaconda3\envs\lamda_ssl\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:1109: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\Users\inima\anaconda3\envs\lamda_ssl\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:696: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  ConvergenceWarning,


In [64]:
print("Accuracy: "+str(accuracy_score(Y_test_SMOTE, y_pred)))
print("F1 score: "+str(f1_score(Y_test_SMOTE, y_pred)))
print("Kappa: "+str(cohen_kappa_score(Y_test_SMOTE, y_pred)))
print("Confusion matrix:")
confusion_matrix(Y_test_SMOTE, y_pred)

Accuracy: 0.7840533696672108
F1 score: 0.014431538190777896
Kappa: 0.010198785366565932
Confusion matrix:


array([[  123,    45],
       [16755, 60874]], dtype=int64)

Linear Discriminant Analysis

In [65]:
lda = LinearDiscriminantAnalysis()
lda.fit(X_train_SMOTE_Amat, Y_train_SMOTE)
y_pred = lda.predict(X_test_SMOTE_Amat)

C:\Users\inima\anaconda3\envs\lamda_ssl\lib\site-packages\sklearn\utils\validation.py:993: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


In [66]:
print("Accuracy: "+str(accuracy_score(Y_test_SMOTE, y_pred)))
print("F1 score: "+str(f1_score(Y_test_SMOTE, y_pred)))
print("Kappa: "+str(cohen_kappa_score(Y_test_SMOTE, y_pred)))
print("Confusion matrix:")
confusion_matrix(Y_test_SMOTE, y_pred)

Accuracy: 0.673252181960744
F1 score: 0.009507481296758106
Kappa: 0.005239301065588342
Confusion matrix:


array([[  122,    46],
       [25374, 52255]], dtype=int64)

## 4.4) Estudio con todo

K Nearest Neighbours

In [67]:
knn = KNeighborsClassifier()
knn.fit(X_train_SMOTE, Y_train_SMOTE)
y_pred = knn.predict(X_test_SMOTE)

C:\Users\inima\anaconda3\envs\lamda_ssl\lib\site-packages\sklearn\neighbors\_classification.py:198: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return self._fit(X, y)


In [68]:
print("Accuracy: "+str(accuracy_score(Y_test_SMOTE, y_pred)))
print("F1 score: "+str(f1_score(Y_test_SMOTE, y_pred)))
print("Kappa: "+str(cohen_kappa_score(Y_test_SMOTE, y_pred)))
print("Confusion matrix:")
confusion_matrix(Y_test_SMOTE, y_pred)

Accuracy: 0.9653199994858414
F1 score: 0.04731638418079096
Kappa: 0.04343010399861447
Confusion matrix:


array([[   67,   101],
       [ 2597, 75032]], dtype=int64)

Random Forest

In [69]:
rf = RandomForestClassifier()
rf.fit(X_train_SMOTE, Y_train_SMOTE)
y_pred = rf.predict(X_test_SMOTE)

C:\Users\inima\anaconda3\envs\lamda_ssl\lib\site-packages\ipykernel_launcher.py:2: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  


In [70]:
print("Accuracy: "+str(accuracy_score(Y_test_SMOTE, y_pred)))
print("F1 score: "+str(f1_score(Y_test_SMOTE, y_pred)))
print("Kappa: "+str(cohen_kappa_score(Y_test_SMOTE, y_pred)))
print("Confusion matrix:")
confusion_matrix(Y_test_SMOTE, y_pred)

Accuracy: 0.9958481689525303
F1 score: 0.3394683026584867
Kappa: 0.3375902866378083
Confusion matrix:


array([[   83,    85],
       [  238, 77391]], dtype=int64)

Decision Forest

In [71]:
clf = tree.DecisionTreeClassifier()
clf.fit(X_train_SMOTE, Y_train_SMOTE)
y_pred = clf.predict(X_test_SMOTE)

In [72]:
print("Accuracy: "+str(accuracy_score(Y_test_SMOTE, y_pred)))
print("F1 score: "+str(f1_score(Y_test_SMOTE, y_pred)))
print("Kappa: "+str(cohen_kappa_score(Y_test_SMOTE, y_pred)))
print("Confusion matrix:")
confusion_matrix(Y_test_SMOTE, y_pred)

Accuracy: 0.9797421494402098
F1 score: 0.09942857142857144
Kappa: 0.09589867499474503
Confusion matrix:


array([[   87,    81],
       [ 1495, 76134]], dtype=int64)

Naive Bayes

In [73]:
gnb = GaussianNB()
gnb.fit(X_train_SMOTE, Y_train_SMOTE)
y_pred = gnb.predict(X_test_SMOTE)

C:\Users\inima\anaconda3\envs\lamda_ssl\lib\site-packages\sklearn\utils\validation.py:993: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


In [74]:
print("Accuracy: "+str(accuracy_score(Y_test_SMOTE, y_pred)))
print("F1 score: "+str(f1_score(Y_test_SMOTE, y_pred)))
print("Kappa: "+str(cohen_kappa_score(Y_test_SMOTE, y_pred)))
print("Confusion matrix:")
confusion_matrix(Y_test_SMOTE, y_pred)

Accuracy: 0.834029589829942
F1 score: 0.01794949802251293
Kappa: 0.013744353492608274
Confusion matrix:


array([[  118,    50],
       [12862, 64767]], dtype=int64)

Multi-layer Perceptron Classifier Neural newtork

In [75]:
mlp = MLPClassifier()
mlp.fit(X_train_SMOTE, Y_train_SMOTE)
y_pred = mlp.predict(X_test_SMOTE)

C:\Users\inima\anaconda3\envs\lamda_ssl\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:1109: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\Users\inima\anaconda3\envs\lamda_ssl\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:696: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  ConvergenceWarning,


In [76]:
print("Accuracy: "+str(accuracy_score(Y_test_SMOTE, y_pred)))
print("F1 score: "+str(f1_score(Y_test_SMOTE, y_pred)))
print("Kappa: "+str(cohen_kappa_score(Y_test_SMOTE, y_pred)))
print("Confusion matrix:")
confusion_matrix(Y_test_SMOTE, y_pred)

Accuracy: 0.8846356543311439
F1 score: 0.02201155061566961
Kappa: 0.01784735355072764
Confusion matrix:


array([[  101,    67],
       [ 8908, 68721]], dtype=int64)

Linear Discriminant Analysis

In [77]:
lda = LinearDiscriminantAnalysis()
lda.fit(X_train_SMOTE, Y_train_SMOTE)
y_pred = lda.predict(X_test_SMOTE)

C:\Users\inima\anaconda3\envs\lamda_ssl\lib\site-packages\sklearn\utils\validation.py:993: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


In [78]:
print("Accuracy: "+str(accuracy_score(Y_test_SMOTE, y_pred)))
print("F1 score: "+str(f1_score(Y_test_SMOTE, y_pred)))
print("Kappa: "+str(cohen_kappa_score(Y_test_SMOTE, y_pred)))
print("Confusion matrix:")
confusion_matrix(Y_test_SMOTE, y_pred)

Accuracy: 0.6362841754823451
F1 score: 0.009382439434252907
Kappa: 0.005110852357214202
Confusion matrix:


array([[  134,    34],
       [28262, 49367]], dtype=int64)